In [1]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
# Importación librerías
import pandas as pd
import os
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
import string
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier


import joblib
import os
os.chdir('..')

In [15]:
os.getcwd()

'C:\\Users\\usuario\\Desktop\\SERGIO\\Master_Data_Analytics_Inteligence\\Subjects\\MasterTrack2_Machine_learning_y_PLN\\MIAD_ML_NLP_2023'

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [5]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [6]:
def clean_text(text):
    text = re.sub((r'[^\w\s]'),'', text).lower() 
    text = re.sub((r'\d+'),'', text).lower()
    text = re.sub((r'_+'),'', text).lower()
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    words = re.sub(r'[^\w\s]', ' ', text).split()
    return ' '.join([wnl.lemmatize(word) for word in words if word not in stopwords])

In [7]:
dataTraining['clean_plot'] = dataTraining['plot'].apply(clean_text)

In [8]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [9]:
tfidf_vectorizer = TfidfVectorizer() #(max_df=0.8, max_features=10000)

In [10]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(dataTraining['clean_plot'], y_genres, test_size=0.20, random_state=9)

In [11]:
xtrain_tfidf = tfidf_vectorizer.fit_transform(X_train)
xtest_tfidf = tfidf_vectorizer.transform(X_test)

In [12]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

# fit model on train data
clf.fit(xtrain_tfidf, y_train_genres)

# make predictions for validation set
y_pred_genres_lr = clf.predict_proba(xtest_tfidf)

In [13]:
roc_auc_score(y_test_genres, y_pred_genres_lr, average='macro')

0.8821186614888777

In [16]:
joblib.dump(clf, 'Proyecto_2/movie_genres_LR.pkl', compress=3)

['Proyecto_2/movie_genres_LR.pkl']

In [ ]:
#python3 Api_movies.py